## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import os, sys
currentdir = os.path.dirname(os.path.realpath("Vacation_Search"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hithadhoo,MV,-0.6000,73.0833,82.08,63,34,4.45,scattered clouds
1,1,Georgetown,MY,5.4112,100.3354,78.73,93,20,3.44,few clouds
2,2,Kirakira,SB,-10.4544,161.9205,83.05,81,100,11.72,overcast clouds
3,3,Deniliquin,AU,-35.5333,144.9667,59.90,85,100,13.69,light rain
4,4,Provideniya,RU,64.3833,-173.3000,3.36,78,75,7.00,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 50


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
17,17,Torbay,CA,47.6666,-52.7314,35.28,95,100,19.57,light rain
32,32,Iisaku,EE,59.1014,27.3081,37.22,96,96,12.97,overcast clouds
36,36,Nizhnevartovsk,RU,60.9344,76.5531,30.22,64,75,7.00,broken clouds
39,39,Liepaja,LV,56.5419,21.0367,40.98,93,75,12.66,light rain
48,48,Ternuvate,UA,47.8305,36.1276,47.48,73,77,12.48,broken clouds
49,49,Severo-Kurilsk,RU,50.6789,156.1250,32.07,74,100,2.48,overcast clouds
53,53,Klaksvik,FO,62.2266,-6.5890,36.52,66,100,35.19,overcast clouds
66,66,Terrace,CA,54.5163,-128.6035,35.51,100,100,2.30,mist
82,82,Saint Anthony,US,45.0205,-93.2180,34.25,84,100,19.57,overcast clouds
83,83,Asau,RO,46.4333,26.4000,45.95,82,98,5.28,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()


City_ID        150
City           150
Country        150
Lat            150
Lng            150
Max Temp       150
Humidity       150
Cloudiness     150
Wind Speed     150
Description    150
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
17,Torbay,CA,35.28,light rain,47.6666,-52.7314,
32,Iisaku,EE,37.22,overcast clouds,59.1014,27.3081,
36,Nizhnevartovsk,RU,30.22,broken clouds,60.9344,76.5531,
39,Liepaja,LV,40.98,light rain,56.5419,21.0367,
48,Ternuvate,UA,47.48,broken clouds,47.8305,36.1276,
49,Severo-Kurilsk,RU,32.07,overcast clouds,50.6789,156.1250,
53,Klaksvik,FO,36.52,overcast clouds,62.2266,-6.5890,
66,Terrace,CA,35.51,mist,54.5163,-128.6035,
82,Saint Anthony,US,34.25,overcast clouds,45.0205,-93.2180,
83,Asau,RO,45.95,overcast clouds,46.4333,26.4000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()     
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))